In [1]:
import numpy as np
import pandas as pd
from Branch import Branch
from sklearn.ensemble import RandomForestClassifier
from ConjunctionSet import ConjunctionSet
from sklearn.tree import export_graphviz
import pydot
import matplotlib.pyplot as plt
from DataPreperation import *
from ReadDatasetFunctions import *
from NewModelBuilder import *
import datetime

%matplotlib inline

In [2]:
data,x_columns,y_column = read_iris_data()
train_x,train_y,test_x,test_y=divide_to_train_test(data,x_columns,y_column)

In [3]:
model=RandomForestClassifier(n_estimators=2,max_depth=2)
model.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=2, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [4]:
cs=ConjunctionSet(x_columns, data, model, amount_of_branches_threshold=1000, filter_approach='probability')

number of branches before filterring: 11
number of branches after filterring: 11
i=0


In [ ]:
for b in cs.conjunctionSet:
    print(b.toString())
    print('')

In [5]:
df=cs.get_conjunction_set_df().round(decimals=5)
df

,0_lower,0_upper,1_lower,1_upper,2_lower,2_upper,3_lower,3_upper,branch_probability,number_of_samples,probas
0,-inf,5.900000,-inf,inf,-inf,2.600000,-inf,0.700000,0.06149,22.00000,"[2.0, 0.0, 0.0]"
1,-inf,5.900000,-inf,inf,2.600000,inf,-inf,0.700000,0.12297,14.83240,"[1.0, 0.9230769230769231, 0.07692307692307693]"
2,5.900000,6.250000,-inf,inf,-inf,inf,-inf,0.700000,0.03556,12.40967,"[1.0, 0.7142857142857143, 0.2857142857142857]"
3,6.250000,inf,-inf,inf,-inf,inf,-inf,0.700000,0.11334,23.45208,"[1.0, 0.125, 0.875]"
4,-inf,5.900000,-inf,inf,-inf,2.600000,0.700000,inf,0.12297,20.44505,"[1.0, 0.90625, 0.09375]"
5,-inf,5.900000,-inf,inf,2.600000,4.750000,0.700000,inf,0.11068,13.78405,"[0.0, 1.8293269230769231, 0.17067307692307693]"
6,5.900000,6.250000,-inf,inf,-inf,4.750000,0.700000,inf,0.04504,11.53256,"[0.0, 1.6205357142857144, 0.3794642857142857]"
7,6.250000,inf,-inf,inf,-inf,4.750000,0.700000,inf,0.14357,21.79449,"[0.0, 1.03125, 0.96875]"
8,-inf,5.900000,-inf,inf,4.750000,inf,0.700000,inf,0.13527,15.16575,"[0.0, 0.9524886877828055, 1.0475113122171946]"
9,5.900000,6.250000,-inf,inf,4.750000,inf,0.700000,inf,0.02608,12.68858,"[0.0, 0.7436974789915967, 1.2563025210084033]"


In [ ]:
df_dict={}
for col in df.columns:
    df_dict[col]=df[col].values

In [ ]:
df

In [ ]:
%%prun
node=Node([True]*len(df))
node.split(df_dict)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
for indx,inst in enumerate(test_x):
    #print('branch:')
    for i,b in enumerate(cs.conjunctionSet):
        if b.containsInstance(inst):
            probas_sum=np.sum(b.label_probas)
            break
    #print(model.predict_proba(inst)[0])
    prediction,depth=node.predict_probas_and_depth(inst,df_dict)
    #print(prediction)
    rf_prediction=model.predict_proba(inst)[0]
    #print(rf_prediction)
    
    if np.argmax(prediction) != np.argmax(rf_prediction):
        print(prediction)
        #print(b.label_probas/probas_sum)
        print(rf_prediction)
        print(test_y[indx])
        print('--------------------')

In [ ]:
def get_depth(t,indx):
    if t.feature[indx] == -2:
        return 1
    return 1 + max(get_depth(t,t.children_left[indx]),get_depth(t,t.children_right[indx]))

In [ ]:
get_depth(t,0)

In [ ]:
est=model.estimators_[0]
t = est.tree_
t.children_left

In [ ]:
import warnings
warnings.filterwarnings('ignore')
for indx,inst in enumerate(test_x):
    #print('branch:')
    for i,b in enumerate(cs.conjunctionSet):
        if b.containsInstance(inst):
            probas_sum=np.sum(b.label_probas)
            #print(b.label_probas/probas_sum)
    #print(model.predict_proba(inst)[0])
    prediction,depth=node.predict_probas_and_depth(inst,df_dict)
    #print(prediction)
    rf_prediction=model.predict_proba(inst)[0]
    #print(rf_prediction)
    
    if np.argmax(prediction) != np.argmax(rf_prediction):
        print(prediction)
        print(rf_prediction)
        print(test_y[indx])
        print('--------------------')

In [ ]:
n=node.right.right.right.right.right.right.right.right
n.get_node_prediction(df_dict)

In [ ]:
#df[node.right.right.right.right.left.right.right.get_node_prediction(df_dict)]
v=df_dict['probas'][node.right.right.right.right.left.right.right.mask][0]
v=[i/np.sum(v) for i in v]
np.array(v)

In [ ]:
node.left.right.left.left.check_feature_split_value(df,'1_lower')

In [ ]:
n=node.left.right.left.left
feature_columns=[col for col in df.columns if 'lower' in col or 'upper' in col]
relevant_cols=[col for col in feature_columns if len(set(df[n.mask][col])) > 1]
relevant_cols

In [ ]:
n=node.left.right.left.left
col='1_lower'
threshold_to_devision_value={}
if 'upper' in col:
    for threshold in set(df[n.mask][col]):
        right_size = len([i for i in df[n.mask][col.replace('upper','lower')] if i > threshold])
        left_size = len(df[n.mask]) - right_size
        threshold_to_devision_value[threshold]=np.abs(left_size-right_size)
else:
    for threshold in set(df[n.mask][col]):
        right_size = len([i for i in df[n.mask][col.replace('lower','upper')] if i <= threshold])
        left_size = len(df[n.mask]) - right_size
        threshold_to_devision_value[threshold] = np.abs(left_size - right_size)
threshold = min(threshold_to_devision_value, key=threshold_to_devision_value.get)
val= threshold_to_devision_value[threshold]
print(threshold,val)

In [ ]:
for inst in test_x:
    for i,b in enumerate(cs.conjunctionSet):
        if b.containsInstance(inst):
            probas_sum=np.sum(b.label_probas)
            print(b.label_probas/probas_sum)
            print(i)
    #print(model.predict_proba(inst)[0])
    print(node.predict(inst,df))
    print('--------------------')

In [ ]:
node.predict(test_x[11],df)

In [ ]:
df

In [ ]:
model.predict_proba(test_x[1])

In [ ]:
model.classes_

In [ ]:
x=np.array([0,1,2])
x+0.001

In [ ]:
model.predict_proba(test_x[1])

In [ ]:
node.predict(test_x[1],df)

In [ ]:
df.sort_values('branch_probability',ascending=False)

In [ ]:
v=np.array([np.array(l) for l in df['probas']])
branches_probas=np.array(df['branch_probability'].values).reshape(len(df),1)
v=v*branches_probas
v=v.mean(axis=0)
v=[i/sum(v) for i in v]
v

In [ ]:
df[node.left.left.left.left.left.left.left.mask].shape

In [ ]:
df[node.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.mask]

In [ ]:
n=node.left.left.left.left.left.left.select_random_split_feature(df,[col for col in df.columns if '_' in col])

In [ ]:
n=node.left.left.left.left.left

In [ ]:
for col in df.columns:
    if 'lower' not in col:
        continue
    

In [ ]:
print(node.left.split_feature)
print(node.left.split_value)

In [ ]:
df[node.left.left.mask]

In [ ]:
print(node.left.left.split_feature)
print(node.left.left.split_value)

In [ ]:
df[node.left.left.left.mask]

In [ ]:
print(node.left.left.left.split_feature)
print(node.left.left.left.split_value)

In [ ]:
original_mask=node.left.left.left.mask
left_mask=node.left.left.left.left_mask
for i,j,k,h in zip(original_mask,left_mask,list(np.logical_and(original_mask,left_mask)),original_mask and left_mask):
    print(i)
    print(j)
    print(k)
    print(h)
    print('-------------')

In [ ]:
node.left.left.left.left_mask

In [ ]:
for i in t.value:
    print(np.argmax(i[0]))

In [ ]:
model=new_model_builder.model
t=model.tree_
new_tree_features=new_model_builder.feature_names
new_features=[]
new_thresholds=[]
original_features=[(int(i.split('<')[0]),float(i.split('<')[1])) for i in new_tree_features]
for feature,threshold in zip(t.feature,t.threshold):
    if feature<0:
        new_features.append(feature)
        new_thresholds.append(threshold)
        continue
    new_features.append(original_features[feature][0])
    new_thresholds.append(original_features[feature][1])
new_thresholds

In [ ]:
records=[]
for b in mg.conjunctionSet:
    records.extend(b.get_branch_records())


In [ ]:
data=pd.DataFrame(records).fillna(0)

In [ ]:
est=rf.estimators_[0]
t=est.tree_
export_graphviz(t,out_file='tree.dot',feature_names=iris.feature_names,class_names=iris.target_names,  
                         filled=True, rounded=True)